# Submission

Arma `submission.csv` para subir a Kaggle.

In [1]:
import sys
sys.path.append('..')

In [2]:
from tqdm import tqdm

from lib.data import load_all_invoices, load_extracted_samples
from lib.details import find_detail_lines
from lib.invoice import find_invoice
from lib.line_classification import is_professional

In [3]:
all_invoices = load_all_invoices()
len(all_invoices)

8411

In [4]:
samples = load_extracted_samples()
len(samples)

8411

In [5]:
with open("../tests/professional_lines.txt") as f:
    truth_prof_lines = f.readlines()
with open("../tests/non_professional_lines.txt") as f:
    truth_non_prof_lines = f.readlines()

len(truth_prof_lines), len(truth_non_prof_lines)

(447, 660)

In [6]:
invoice_dict = {}
for invoice in all_invoices:
    invoice_dict[invoice] = None

for sample in tqdm(samples):
    filename = sample["filename"][:-4] # .pdf
    
    invoice = find_invoice(sample)
    detail_lines = find_detail_lines(sample)
    
    total = 0
    for line in detail_lines:
        if line is not None:
            if line in truth_non_prof_lines:
                continue # skip
            if line in truth_prof_lines or is_professional(line["desc"]):
                total += line["total"]

    if total == 0:
        total = None

    if invoice is not None and total is not None:
        invoice_dict[invoice] = total

100%|██████████| 8411/8411 [03:57<00:00, 35.47it/s]


In [7]:
all_totals = [total for total in invoice_dict.values() if total is not None]
avg = sum(all_totals) / len(all_totals)
avg = round(avg, 2)
median = sorted(all_totals)[len(all_totals) // 2]
median = round(median, 2)
len(all_totals), len(all_totals) / len(samples), avg, median

(7994, 0.9504220663416953, 12349.48, 9048.57)

In [8]:
# write csv
with open("submission.csv", 'w') as f:
    f.write('Invoice Number,Total Charged\n')
    for invoice, total in invoice_dict.items():

        if total == None:
            # https://en.wikipedia.org/wiki/Mean_absolute_error#Proof_of_optimality
            total = median

        f.write(f'{invoice},{total}\n')

In [9]:
import pandas as pd

df = pd.read_csv("../submissions/38_refactors_and_fixes.csv")
df[df["Total Charged"] != 9783.46]["Total Charged"].median()

8779.04